In [59]:
#TASK 1 - Please download dataset or generate it using the attached script dataset_generator.py
import pandas as pd
from tqdm import tqdm
import csv
import random
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

random.seed(1)

letters = string.ascii_lowercase
letters_upper = string.ascii_uppercase
for _i in range(0, 10):
    letters += letters

for _i in range(0, 10):
    letters += letters_upper


def random_string(stringLength=10):
    """Generate a random string of fixed length """
    return ''.join(random.sample(letters, stringLength))


print("Products between {} and {}".format(1, 100000))
product_ids = [x for x in range(1, 100000)]
dates = ['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10']
seller_ids = [x for x in range(1, 10)]


#   Generate products
products = [[0, "product_0", 22]]
for p in tqdm(product_ids):
    products.append([p, "product_{}".format(p), random.randint(1, 150)])
#   Save dataframe
df = pd.DataFrame(products)
df.columns = ["product_id", "product_name", "price"]
df.to_csv("products.csv", index=False)
del df
del products

#   Generate sellers
sellers = [[0, "seller_0", 100000]]
for s in tqdm(seller_ids):
    sellers.append([s, "seller_{}".format(s), random.randint(0, 100000)])
#   Save dataframe
df = pd.DataFrame(sellers)
df.columns = ["seller_id", "seller_name", "daily_target"]
df.to_csv("sellers.csv", index=False)

#   Generate sales
total_rows = 100000
prod_zero = int(total_rows * 0.95)
prod_others = total_rows - prod_zero + 1
df_array = [["order_id", "product_id", "seller_id", "date", "num_pieces_sold", "bill_raw_text"]]
with open('sales.csv', 'w', newline='') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerows(df_array)

order_id = 0
for i in tqdm(range(0, 40)):
    df_array = []

    for i in range(0, prod_zero):
        order_id += 1
        df_array.append([order_id, 0, 0, random.choice(dates), random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

    df_array = []
    for i in range(0, prod_others):
        order_id += 1
        df_array.append(
            [order_id, random.choice(product_ids), random.choice(seller_ids), random.choice(dates),
             random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

print("Done")

Products between 1 and 100000


100%|██████████| 40/40 [19:04<00:00, 28.61s/it]

Done


In [61]:
#-------------------------TASK 1----------------------------------------------
#importing SparkSession class
from pyspark.sql import SparkSession

In [62]:
#assigning config to spark variable
spark=SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option","some-value") \
.getOrCreate()

In [63]:
#creating sparkcontext
sc=spark.sparkContext

In [64]:
#2.1. Please upload data from sellers.csv into sellers_rdd variable. Use the textFile method with map function: 
sellers_rdd=sc.textFile("/home/jovyan/work/sellers.csv").map(lambda x:x.split(","))

In [65]:
#2.2. Take first 10 elements of sellers_rdd.
sellers_rdd.take(10)

[['seller_id', 'seller_name', 'daily_target'],
 ['0', 'seller_0', '100000'],
 ['1', 'seller_1', '83478'],
 ['2', 'seller_2', '94114'],
 ['3', 'seller_3', '50299'],
 ['4', 'seller_4', '72654'],
 ['5', 'seller_5', '28862'],
 ['6', 'seller_6', '61878'],
 ['7', 'seller_7', '72047'],
 ['8', 'seller_8', '54715']]

In [66]:
#2.3. Save the first element of sellers_rdd into sellers_rdd_header variable
sellers_rdd_header=sellers_rdd.top(1)

In [67]:
#2.4. Filter the sellers_rdd excluding the sellers_rdd_header value, and reassign it to sellers_rdd. 
sellers_rdd=sellers_rdd.filter(lambda x:x not in sellers_rdd_header)

In [68]:
#2.5.2. Take top 3 elements of sellers_rdd in ascending order.
sellers_rdd.sortByKey(True,1).take(3)

[['0', 'seller_0', '100000'],
 ['1', 'seller_1', '83478'],
 ['2', 'seller_2', '94114']]

In [69]:
#2.5.1. Take top 3 elements of sellers_rdd in descending order.
sellers_rdd.sortByKey(False,1).take(3)

[['9', 'seller_9', '82824'],
 ['8', 'seller_8', '54715'],
 ['7', 'seller_7', '72047']]

In [70]:
#2.6. Get a number of partitions of sellers_rdd.
sellers_rdd.getNumPartitions()

2

In [71]:
#---------------------TASK 3--------------------------------------
#3.1. Please create the calendar_rdd variable containing years starting from 1970 to 2099 using parallelized collections. 
calendar_rdd=sc.parallelize(range(1970,2100))

In [77]:
#3.2. Filter the calendar_rdd data containing only coming years (2022, 2023..) and save it to the calendar_coming_years_rdd variable
import datetime
calendar_coming_years_rdd=calendar_rdd.filter(lambda x:int(datetime.datetime.today().strftime('%Y'))<=x)

In [78]:
calendar_coming_years_rdd.collect()

[2023,
 2024,
 2025,
 2026,
 2027,
 2028,
 2029,
 2030,
 2031,
 2032,
 2033,
 2034,
 2035,
 2036,
 2037,
 2038,
 2039,
 2040,
 2041,
 2042,
 2043,
 2044,
 2045,
 2046,
 2047,
 2048,
 2049,
 2050,
 2051,
 2052,
 2053,
 2054,
 2055,
 2056,
 2057,
 2058,
 2059,
 2060,
 2061,
 2062,
 2063,
 2064,
 2065,
 2066,
 2067,
 2068,
 2069,
 2070,
 2071,
 2072,
 2073,
 2074,
 2075,
 2076,
 2077,
 2078,
 2079,
 2080,
 2081,
 2082,
 2083,
 2084,
 2085,
 2086,
 2087,
 2088,
 2089,
 2090,
 2091,
 2092,
 2093,
 2094,
 2095,
 2096,
 2097,
 2098,
 2099]

In [74]:
#3.3. Filter the calendar_rdd data containing only leap years (1972…2000…2016, 2020, 2024, …) and save it to the calendar_leap_years_rdd variable.
calendar_leap_years_rdd=calendar_rdd.filter(lambda x:x%4==0)

In [76]:
calendar_leap_years_rdd.collect()

[1972,
 1976,
 1980,
 1984,
 1988,
 1992,
 1996,
 2000,
 2004,
 2008,
 2012,
 2016,
 2020,
 2024,
 2028,
 2032,
 2036,
 2040,
 2044,
 2048,
 2052,
 2056,
 2060,
 2064,
 2068,
 2072,
 2076,
 2080,
 2084,
 2088,
 2092,
 2096]

In [79]:
#3.4. Create calendar_not_leap_years_rdd using some of SET operator and calendar_rdd and calendar_leap_years_rdd. It should contain all not leap years. Sort all values in ascending order and collect them.
calendar_non_leap_years_rdd=calendar_rdd.subtract(calendar_leap_years_rdd).sortBy(lambda x:x)

In [81]:
calendar_non_leap_years_rdd.take(100)

[1970,
 1971,
 1973,
 1974,
 1975,
 1977,
 1978,
 1979,
 1981,
 1982,
 1983,
 1985,
 1986,
 1987,
 1989,
 1990,
 1991,
 1993,
 1994,
 1995,
 1997,
 1998,
 1999,
 2001,
 2002,
 2003,
 2005,
 2006,
 2007,
 2009,
 2010,
 2011,
 2013,
 2014,
 2015,
 2017,
 2018,
 2019,
 2021,
 2022,
 2023,
 2025,
 2026,
 2027,
 2029,
 2030,
 2031,
 2033,
 2034,
 2035,
 2037,
 2038,
 2039,
 2041,
 2042,
 2043,
 2045,
 2046,
 2047,
 2049,
 2050,
 2051,
 2053,
 2054,
 2055,
 2057,
 2058,
 2059,
 2061,
 2062,
 2063,
 2065,
 2066,
 2067,
 2069,
 2070,
 2071,
 2073,
 2074,
 2075,
 2077,
 2078,
 2079,
 2081,
 2082,
 2083,
 2085,
 2086,
 2087,
 2089,
 2090,
 2091,
 2093,
 2094,
 2095,
 2097,
 2098,
 2099]

In [82]:
#3.5. Group values of calendar_rdd by millennium (use the 2000 value for grouping).
calendar_rdd.groupBy(lambda x: str(x)[0:1]).mapValues(list).collect()

[('1',
  [1970,
   1971,
   1972,
   1973,
   1974,
   1975,
   1976,
   1977,
   1978,
   1979,
   1980,
   1981,
   1982,
   1983,
   1984,
   1985,
   1986,
   1987,
   1988,
   1989,
   1990,
   1991,
   1992,
   1993,
   1994,
   1995,
   1996,
   1997,
   1998,
   1999]),
 ('2',
  [2000,
   2001,
   2002,
   2003,
   2004,
   2005,
   2006,
   2007,
   2008,
   2009,
   2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2018,
   2019,
   2020,
   2021,
   2022,
   2023,
   2024,
   2025,
   2026,
   2027,
   2028,
   2029,
   2030,
   2031,
   2032,
   2033,
   2034,
   2035,
   2036,
   2037,
   2038,
   2039,
   2040,
   2041,
   2042,
   2043,
   2044,
   2045,
   2046,
   2047,
   2048,
   2049,
   2050,
   2051,
   2052,
   2053,
   2054,
   2055,
   2056,
   2057,
   2058,
   2059,
   2060,
   2061,
   2062,
   2063,
   2064,
   2065,
   2066,
   2067,
   2068,
   2069,
   2070,
   2071,
   2072,
   2073,
   2074,
   2075,
   2076,
   2077,
   2078,
   

In [83]:
#3.6. Imagine that mistakenly you deleted calendar_rdd using command “del calendar_rdd”. Please try to get the same data using UNION command and save it to the new_calendar_rdd.
new_calendar_rdd=calendar_non_leap_years_rdd.union(calendar_leap_years_rdd)

In [84]:
#3.7. You want to make sure that the UNION command above didn’t produce any duplicated data. Print count of elements of new_calendar_rdd and then a count of distinct values.
#Code to verify the duplicates
l=[]
for i,j in new_calendar_rdd.countByValue().items():
    if j>1:
        a=j+1
        print('Duplicate exists for key-> '+str(i)+' number of values-> '+str(j))
    else:
        l.append('No duplicates')
print(set(l))

{'No duplicates'}


In [86]:
#Print count of elements of new_calendar_rdd and then a count of distinct values.
new_calendar_rdd.count()
new_calendar_rdd.countByValue()

defaultdict(int,
            {1970: 1,
             1971: 1,
             1973: 1,
             1974: 1,
             1975: 1,
             1977: 1,
             1978: 1,
             1979: 1,
             1981: 1,
             1982: 1,
             1983: 1,
             1985: 1,
             1986: 1,
             1987: 1,
             1989: 1,
             1990: 1,
             1991: 1,
             1993: 1,
             1994: 1,
             1995: 1,
             1997: 1,
             1998: 1,
             1999: 1,
             2001: 1,
             2002: 1,
             2003: 1,
             2005: 1,
             2006: 1,
             2007: 1,
             2009: 1,
             2010: 1,
             2011: 1,
             2013: 1,
             2014: 1,
             2015: 1,
             2017: 1,
             2018: 1,
             2019: 1,
             2021: 1,
             2022: 1,
             2023: 1,
             2025: 1,
             2026: 1,
             2027: 1,
             20

In [87]:
#3.8. Imagine that the time machine was created, and you are transported 1000 years ahead. Apply a map function to each element of calendar_rdd and create a calendar_rdd_future.   
calendar_rdd_future=calendar_rdd.map(lambda x:x+1000)

In [88]:
calendar_rdd_future.collect()

[2970,
 2971,
 2972,
 2973,
 2974,
 2975,
 2976,
 2977,
 2978,
 2979,
 2980,
 2981,
 2982,
 2983,
 2984,
 2985,
 2986,
 2987,
 2988,
 2989,
 2990,
 2991,
 2992,
 2993,
 2994,
 2995,
 2996,
 2997,
 2998,
 2999,
 3000,
 3001,
 3002,
 3003,
 3004,
 3005,
 3006,
 3007,
 3008,
 3009,
 3010,
 3011,
 3012,
 3013,
 3014,
 3015,
 3016,
 3017,
 3018,
 3019,
 3020,
 3021,
 3022,
 3023,
 3024,
 3025,
 3026,
 3027,
 3028,
 3029,
 3030,
 3031,
 3032,
 3033,
 3034,
 3035,
 3036,
 3037,
 3038,
 3039,
 3040,
 3041,
 3042,
 3043,
 3044,
 3045,
 3046,
 3047,
 3048,
 3049,
 3050,
 3051,
 3052,
 3053,
 3054,
 3055,
 3056,
 3057,
 3058,
 3059,
 3060,
 3061,
 3062,
 3063,
 3064,
 3065,
 3066,
 3067,
 3068,
 3069,
 3070,
 3071,
 3072,
 3073,
 3074,
 3075,
 3076,
 3077,
 3078,
 3079,
 3080,
 3081,
 3082,
 3083,
 3084,
 3085,
 3086,
 3087,
 3088,
 3089,
 3090,
 3091,
 3092,
 3093,
 3094,
 3095,
 3096,
 3097,
 3098,
 3099]

In [90]:
#3.9. Print the statistic of calendar_rdd data (count, min, max..)
calendar_rdd.stats()

(count: 130, mean: 2034.5, stdev: 37.526657191921586, max: 2099.0, min: 1970.0)